In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf

tf.random.set_seed(seed)

os.environ['PATH'] = os.environ['XILINX_VITIS'] + '/bin:' + os.environ['PATH']

2025-04-15 12:00:47.152319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-15 12:00:47.164032: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-15 12:00:47.167482: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 12:00:47.176732: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-15 12:00:48.112434: W tensorflow/compiler/tf2

# Data Preprocessing

In [2]:
import awkward as ak
W_data = ak.from_parquet("W_data.parquet")
Z_data = ak.from_parquet("Z_data.parquet")

In [3]:
# add labels
W_data["MET"] = len(W_data)*[1]
Z_data["MET"] = len(Z_data)*[0]

In [4]:
# combine and shuffle
combined = ak.concatenate([W_data, Z_data])
combined = combined[ak.argsort(np.random.rand(len(combined)))]

In [5]:
for field in ["jet_pt", "jet_eta", "jet_phi"]:
    combined = ak.with_field(combined, combined[field][:, :3], field)
    combined[field] = ak.fill_none(ak.pad_none(combined[field],3, axis = 1),0)

for field in ["lep_pt", "lep_eta", "lep_phi"]:
    combined = ak.with_field(combined, combined[field][:, :2], field)
    combined[field] = ak.fill_none(ak.pad_none(combined[field],2, axis = 1),0)

In [14]:
combined["jet_pt"]

<Array [[0, 0, 0], [0, ..., 0], ..., [0, 0, 0]] type='101060956 * var * flo...'>

In [6]:
indices = np.arange(len(combined))
train_idx, test_idx = train_test_split(indices, stratify=combined["MET"], test_size = 0.3)

In [7]:
train = combined[train_idx]
test = combined[test_idx]

In [8]:
y_train = ak.to_numpy(train["MET"])
y_test = ak.to_numpy(test["MET"])

x_train = np.stack([
    ak.to_numpy(train["jet_pt"][:, 0]),
    ak.to_numpy(train["jet_eta"][:, 0]),
    ak.to_numpy(train["jet_phi"][:, 0]),
    ak.to_numpy(train["jet_pt"][:, 1]),
    ak.to_numpy(train["jet_eta"][:, 1]),
    ak.to_numpy(train["jet_phi"][:, 1]),
    ak.to_numpy(train["jet_pt"][:, 2]),
    ak.to_numpy(train["jet_eta"][:, 2]),
    ak.to_numpy(train["jet_phi"][:, 2]),
    ak.to_numpy(train["lep_pt"][:, 0]),
    ak.to_numpy(train["lep_eta"][:, 0]),
    ak.to_numpy(train["lep_phi"][:, 0]),
    ak.to_numpy(train["lep_pt"][:, 1]),
    ak.to_numpy(train["lep_eta"][:, 1]),
    ak.to_numpy(train["lep_phi"][:, 1]),
    ak.to_numpy(train["met_et"]),
    ak.to_numpy(train["met_phi"])
], axis=1)

# Constructing Model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

model = Sequential()
model.add(Dense(64, input_shape=(17,), name='fc1', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu1'))
model.add(Dense(32, name='fc2', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu2'))
model.add(Dense(32, name='fc3', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu3'))
model.add(Dense(5, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='softmax', name='softmax'))

2025-04-15 12:04:15.608146: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38148 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:27:00.0, compute capability: 8.0


In [20]:
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
model.fit(x_train, y_train, batch_size = 1024, epochs = 10, validation_split = 0.25)

Epoch 1/10


ValueError: in user code:

    File "/home/frodriguez/.conda/envs/hls4ml-tutorial/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/frodriguez/.conda/envs/hls4ml-tutorial/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/frodriguez/.conda/envs/hls4ml-tutorial/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/home/frodriguez/.conda/envs/hls4ml-tutorial/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1148, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/frodriguez/.conda/envs/hls4ml-tutorial/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1206, in compute_loss
        return self.

In [27]:
x_train

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  4.57703281e+04, -2.31341910e+00, -7.48141587e-01,
        4.22642891e+04, -1.75426650e+00,  1.85225403e+00,  2.20487051e+04,
       -2.70963883e+00])